In [1]:
import pickle
with open('train_sentence_transformer.pkl', 'rb') as f:
    train = pickle.load(f)
with open('val_sentence_transformer.pkl', 'rb') as f:
    val = pickle.load(f)
with open('test_sentence_transformer.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
import json
with open('test.json', encoding = 'utf8') as json_file:
    data = json.load(json_file)
    
conversations = []
summaries = []
for i in range(0, len(data)):
    if len(data[i]['dialogue'].split('\r\n')) > 1:
        sentences = data[i]['dialogue'].split('\r\n')
    else:
        sentences = data[i]['dialogue'].split('\n')

    conversations.append(sentences)
    summaries.append(data[i]['summary'].strip('\n').replace('\r\nt', ' '))

In [3]:
import numpy as np
length = []
for i in range(0, len(train)):
    length.append(len(train[i]))
for i in range(0, len(val)):
    length.append(len(val[i]))
for i in range(0, len(test)):
    length.append(len(test[i]))
X = []
for i in range(0, len(train)):
    for j in range(0, len(train[i])):
        X.append(np.array(train[i][j]))
for i in range(0, len(val)):
    for j in range(0,len(val[i])):
        X.append(np.array(val[i][j]))
for i in range(0, len(test)):
    for j in range(0, len(test[i])):
        X.append(np.array(test[i][j]))

In [4]:
from hmmlearn import hmm

In [5]:
remodel = hmm.GaussianHMM(n_components=4, n_iter = 50, covariance_type = 'diag', verbose = True, init_params="cm", params="cmts")
remodel.startprob_ = np.array([1, 0.0, 0.0, 0.0])
remodel.transmat_ = np.array([
                 [0.33, 0.34, 0.33, 0],
                 [0.0, 0.33, 0.34, 0.33],
                 [0.0, 0.0, 0.5, 0.5],
                 [0.0, 0.0, 0.0, 1.0]]
                            )

In [6]:
remodel.startprob_

array([1., 0., 0., 0.])

In [7]:
remodel.transmat_

array([[0.33, 0.34, 0.33, 0.  ],
       [0.  , 0.33, 0.34, 0.33],
       [0.  , 0.  , 0.5 , 0.5 ],
       [0.  , 0.  , 0.  , 1.  ]])

In [8]:
remodel.fit(X, length)

         1 -808009.18669734             +nan
         2 -791828.07089089  +16181.11580645
         3 -791131.08941758    +696.98147331
         4 -790829.86423684    +301.22518074
         5 -790737.42377778     +92.44045906
         6 -790726.49761171     +10.92616606
         7 -790719.19829006      +7.29932165
         8 -790713.74423593      +5.45405413
         9 -790713.73789898      +0.00633695


GaussianHMM(init_params='cm', n_components=4, n_iter=50, params='cmts',
            verbose=True)

In [9]:
remodel.transmat_

array([[0.81544096, 0.11241699, 0.07214204, 0.        ],
       [0.        , 0.78028379, 0.15910937, 0.06060684],
       [0.        , 0.        , 0.72871955, 0.27128045],
       [0.        , 0.        , 0.        , 1.        ]])

In [10]:
remodel.startprob_

array([1., 0., 0., 0.])

In [11]:
def encode_convs(profix):
    sent_label = []
    with open(profix + '_sentence_transformer.pkl', 'rb') as f:
        data = pickle.load(f)
    for i in range(0, len(data)):
   
        labels = remodel.decode(np.array(data[i]))[1]
        sent_label.append(labels)
    
    with open(profix + '_sent_trans_cons_label.pkl', 'wb') as f:
        pickle.dump(sent_label, f)
    return sent_label

In [12]:
l = encode_convs('train')

In [13]:
l = encode_convs('val')

In [14]:
l = encode_convs('test')